# Data Preprocessing

In [7]:
import csv
csv.writer(open("Data/NER_nlp_corpus.tsv", 'w+'), delimiter='\t').writerows(csv.reader(open("Data/ner_dataset_nlp_corpus.csv")))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 2588: character maps to <undefined>

In [10]:
# Convert .tsv file to dataturks json format. 
import json
import logging
import sys
def tsv_to_json_format(input_path,output_path,unknown_label):
    try:
        f=open(input_path,'r') # input file
        fp=open(output_path, 'w') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        for line in f:
            if line[0:len(line)-1]!='.\tO':
                print("=====Line======", line)
                word,entity=line.split('\t')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None
tsv_to_json_format("Data/NER_DATA_corpus.tsv",'Data/ner_corpus_260.json','abc')


ERROR:root:Unable to process file
error = too many values to unpack (expected 2)
Traceback (most recent call last):
  File "<ipython-input-10-e8dd5f68b49d>", line 17, in tsv_to_json_format
    word,entity=line.split('\t')
ValueError: too many values to unpack (expected 2)


=====Line====== Word	Tag

=====Line====== Thousands	O

=====Line====== of	O

=====Line====== demonstrators	O

=====Line====== have	O

=====Line====== marched	O

=====Line====== through	O

=====Line====== London	B-geo

=====Line====== to	O

=====Line====== protest	O

=====Line====== the	O

=====Line====== war	O

=====Line====== in	O

=====Line====== Iraq	B-geo

=====Line====== and	O

=====Line====== demand	O

=====Line====== the	O

=====Line====== withdrawal	O

=====Line====== of	O

=====Line====== British	B-gpe

=====Line====== troops	O

=====Line====== from	O

=====Line====== that	O

=====Line====== country	O

=====Line====== Families	O

=====Line====== of	O

=====Line====== soldiers	O

=====Line====== killed	O

=====Line====== in	O

=====Line====== the	O

=====Line====== conflict	O

=====Line====== joined	O

=====Line====== the	O

=====Line====== protesters	O

=====Line====== who	O

=====Line====== carried	O

=====Line====== banners	O

=====Line====== with	O

=====Line====== such	O



In [2]:
# Convert json file to spaCy format.
import plac
import logging
import argparse
import sys
import os
import json
import pickle

@plac.annotations(input_file=("Input file", "option", "i", str), output_file=("Output file", "option", "o", str))

def main(input_file=None, output_file=None):
    try:
        training_data = []
        lines=[]
        with open(input_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        print(training_data)

        with open(output_file, 'wb') as fp:
            pickle.dump(training_data, fp)

    except Exception as e:
        logging.exception("Unable to process " + input_file + "\n" + "error = " + str(e))
        return None
if __name__ == '__main__':
    plac.call(main)

usage: ipykernel_launcher.py [-h] [-i None] [-o None]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\sadam\AppData\Roaming\jupyter\runtime\kernel-ef984d21-f439-430a-ab6d-c28964631bdb.json


SystemExit: 2

C:\Users\sadam\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
filename = r"Data/ner_dataset_nlp_corpus.csv"
with open(filename, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    fields = csvreader.next()
    fields

AttributeError: '_csv.reader' object has no attribute 'next'